# Amazon SageMaker Model Monitor


This notebook shows how to:

- Amazon SageMaker에서 머신 러닝 모델을 호스팅하고 추론 요청, 결과 및 메타데이터를 캡처하는 방법
- 훈련 데이터셋을 분석하여 기본 제약 조건을 생성하는 방법
- 라이브 엔드포인트를 모니터링하여 제약 조건 위반을 감지하는 방법

---

## Background

Amazon SageMaker는 배포된 모델의 호출에 대한 입력, 출력 및 메타데이터를 캡처할 수 있게 해줍니다. 데이터를 분석하고 품질을 모니터링할 수 있는 기능도 제공합니다. 이에 대한 실습을 진행해보겠습니다.

## Runtime

This notebook uses an hourly monitor, so it takes between 30-90 minutes to run.

## Contents

1. [파트 A: Amazon SageMaker 엔드포인트에서 실시간 추론 데이터 캡처](#PART-A:-Capturing-real-time-inference-data-from-Amazon-SageMaker-endpoints)
1. [파트 B: 모델 모니터 - 기준 설정 및 지속적인 모니터링](#PART-B:-Model-Monitor---Baselining-and-continuous-monitoring)
   1. [Constraint suggestion with baseline/training dataset](#1.-Constraint-suggestion-with-baseline/training-dataset)
   1. [Analyze collected data for data quality issues](#2.-Analyze-collected-data-for-data-quality-issues)

---

## Setup

To get started, make sure you have these prerequisites completed:

- Specify an AWS Region to host your model.
- An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3).
- Use the default S3 bucket to store the data used to train your model, any additional model data, and the data captured from model invocations. For demonstration purposes, you are using the same bucket for these. In reality, you might want to separate them with different security policies.


In [1]:
import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role, session

sm_session = sagemaker.Session()
region = sm_session.boto_region_name

role = get_execution_role()
print("Role ARN: {}".format(role))

bucket = sm_session.default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = "sagemaker/DEMO-ModelMonitor"

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)
code_prefix = "{}/code".format(prefix)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "postprocessor.py")

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Role ARN: arn:aws:iam::672381912436:role/service-role/AmazonSageMaker-ExecutionRole-20240723T040833
Demo Bucket: sagemaker-ap-northeast-2-672381912436
Capture path: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/datacapture
Report path: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/reports
Preproc Code path: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/code/preprocessor.py
Postproc Code path: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/code/postprocessor.py


## PART A: Capturing real-time inference data from Amazon SageMaker endpoints

Create an endpoint to showcase the data capture capability in action.

### Upload the pre-trained model to Amazon S3

이 코드는 배포할 준비가 된 사전 훈련된 XGBoost 모델을 업로드합니다.


In [2]:
model_file = open("model/xgb-churn-prediction-model.tar.gz", "rb")
s3_key = os.path.join(prefix, "xgb-churn-prediction-model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

### Deploy the model to Amazon SageMaker

Start with deploying a pre-trained churn prediction model. Here, you create the model object with the image and model data.


In [3]:
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz".format(
    bucket, region, prefix
)

image_uri = retrieve("xgboost", region, "0.90-1")

model = Model(image_uri=image_uri, model_data=model_url, role=role)

모델 데이터 품질을 모니터링하기 위해 데이터 캡처를 활성화하려면 DataCaptureConfig라는 새로운 캡처 옵션을 지정합니다.
이를 통해서 요청이나 응답에 대한 페이로드를 캡쳐할 수 있습니다.
이후 deployment 단계에서 해봤던 것과 동일하게 .deploy를 사용하려 endpoint 배포를 진행해봅니다.


In [4]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName=DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07
-----!

### Invoke the deployed model

deploy가 완료되었다면 생성된 엔드포인트에 데이터를 보내서 실시간으로 추론을 할 수 있습니다.
이전 단계에서 데이터 캡처를 활성화했기 때문에 요청 및 응답 페이로드와 추가 메타데이터가 DataCaptureConfig에서 지정한 s3위치에 저장되게 됩니다.

이 단계에서는 약 3분 동안 포함된 샘플 데이터를 사용하여 엔드포인트를 호출해서 데이터가 캡쳐되도록 합니다.


This step invokes the endpoint with included sample data for about 3 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.


In [5]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import time

predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

# Get a subset of test data for a quick test
!head -180 test_data/test-dataset-input-cols.csv > test_data/test_sample.csv
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open("test_data/test_sample.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        response = predictor.predict(data=payload)
        time.sleep(1)

print("Done!")

Sending test traffic to the endpoint DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07. 
Please wait...
Done!


### View captured data

Amazon S3에 저장된 데이터 캡처 파일이 어떤게 있는지 가져와보겠습니다.

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`


In [7]:
s3_client = boto3.Session().client("s3")
current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07/AllTraffic/2024/07/22/20/29-10-429-96d4b620-b7f7-4921-9a1b-72800e5b36ee.jsonl
 sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07/AllTraffic/2024/07/22/20/30-10-794-c32f4e48-3cac-4e46-a62b-3b47de95bc10.jsonl
 sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07/AllTraffic/2024/07/22/20/31-10-904-96a29de4-ba16-49c4-962d-5dbc41e67761.jsonl
 sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07/AllTraffic/2024/07/22/20/32-10-982-0ad712d4-9cf8-4ba6-b0c0-3df01dc69b2c.jsonl


한 파일을 열어서 JSON-line형식으로 캡쳐된 데이터를 한 번 출력해봅니다.


In [9]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"108,0,198.5,99,267.8,60,354.9,75,9.4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.03810077905654907","encoding":"CSV"}},"eventMetadata":{"eventId":"6caf6eee-822f-41fe-8506-26c999b32ce2","inferenceTime":"2024-07-22T20:32:10Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"71,0,103.3,103,138.5,79,164.8,98,9.0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.03876146301627159","encoding":"CSV"}},"eventMetadata":{"eventId":"9d73d8e1-9ffa-46d0-8769-58f322542816","inferenceTime":"2

아래의 코드를 실행해서 좀 더 보기쉬운 형태로 확인해보겠습니다.


In [10]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "108,0,198.5,99,267.8,60,354.9,75,9.4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.03810077905654907",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "6caf6eee-822f-41fe-8506-26c999b32ce2",
    "inferenceTime": "2024-07-22T20:32:10Z"
  },
  "eventVersion": "0"
}


다음으로는 Amazon SageMaker가 Amazon S3에서 수집된 데이터를 모니터링하는 방법에 대해 보겠습니다.


## PART B: Model Monitor - Baselining and continuous monitoring


데이터 수집 외에도, Amazon SageMaker는 엔드포인트에서 관찰된 데이터를 모니터링하고 평가할 수 있는 기능을 제공합니다. 이를 위해 다음 단계를 따릅니다:

1. 실시간 트래픽과 비교할 기준선을 설정합니다.
2. 기준선이 준비되면, 기준선과 비교할 수 있도록 지속적으로 평가하고 모니터링할 일정을 설정합니다.


### 1. Constraint suggestion with baseline/training dataset


The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.


In [11]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = "s3://{}/{}".format(bucket, baseline_data_prefix)
baseline_results_uri = "s3://{}/{}".format(bucket, baseline_results_prefix)
print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

Baseline data uri: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/baselining/data
Baseline results uri: s3://sagemaker-ap-northeast-2-672381912436/sagemaker/DEMO-ModelMonitor/baselining/results


In [12]:
training_data_file = open("test_data/training-dataset-with-header.csv", "rb")
s3_key = os.path.join(baseline_prefix, "data", "training-dataset-with-header.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

#### Create a baselining job with training dataset


Now that you have the training data ready in Amazon S3, start a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` starts a `ProcessingJob` using an Amazon SageMaker provided Model Monitor container to generate the constraints.


In [13]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri + "/training-dataset-with-header.csv",
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True,
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2024-07-22-20-46-52-342


............2024-07-22 20:48:49.969461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-22 20:48:49.969491: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-22 20:48:51.537862: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-22 20:48:51.537894: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-22 20:48:51.537919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-117-124.ap-northeast-2.compute.internal): /proc/driver/nvidia/version does not exist
2024-07-22 20:48:

#### Explore the generated constraints and statistics

생성된 기준선 통계 및 제약 조건 탐색


In [14]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
sagemaker/DEMO-ModelMonitor/baselining/results/constraints.json
 sagemaker/DEMO-ModelMonitor/baselining/results/statistics.json


In [ ]:
# 데이터 처리 및 분석을 위해 pandas 라이브러리를 가져옵니다.
import pandas as pd 

# 최근의 기준선 작업(baselining_job)을 가져옵니다. 
baseline_job = my_default_monitor.latest_baselining_job 
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [18]:
import pandas as pd

constraints_df = pd.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head(10) # 생성된 데이터프레임 constraints_df의 상위 10개 행을 출력합니다. 이로써 제안된 제약 조건의 구조와 내용을 확인할 수 있습니다.


,name,inferred_type,completeness,num_constraints.is_non_negative
0,Churn,Integral,1.0,True
1,Account Length,Integral,1.0,True
2,VMail Message,Integral,1.0,True
3,Day Mins,Fractional,1.0,True
4,Day Calls,Integral,1.0,True
5,Eve Mins,Fractional,1.0,True
6,Eve Calls,Integral,1.0,True
7,Night Mins,Fractional,1.0,True
8,Night Calls,Integral,1.0,True
9,Intl Mins,Fractional,1.0,True


### 2. Analyze collected data for data quality issues

When you have collected the data above, analyze and monitor the data with Monitoring Schedules.


#### Create a schedule


In [19]:
# Upload some test scripts to the S3 bucket for pre- and post-processing
bucket = boto3.Session().resource("s3").Bucket(bucket)
bucket.Object(code_prefix + "/preprocessor.py").upload_file("preprocessor.py")
bucket.Object(code_prefix + "/postprocessor.py").upload_file("postprocessor.py")

You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.


In [20]:
# CronExpressionGenerator 모듈을 sagemaker.model_monitor에서 임포트합니다. 
# 이 모듈은 CloudWatch 이벤트를 사용하여 주기적으로 모니터링 작업을 실행하는 크론 표현식을 생성하는 데 사용됩니다.
from sagemaker.model_monitor import CronExpressionGenerator

mon_schedule_name = "DEMO-xgb-churn-pred-model-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

# my_default_monitor 객체를 사용하여 모델 모니터링 스케줄을 생성합니다. 
# create_monitoring_schedule로 주기적으로 모델의 성능을 모니터링하고 결과를 S3 버킷에 저장합니다.
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint, # 모니터링할 모델의 엔드포인트를 지정합니다. 
    # record_preprocessor_script=pre_processor_script,
    post_analytics_processor_script=s3_code_postprocessor_uri, # 분석 후 처리 스크립트의 S3 URI를 지정합니다. 이 스크립트는 모니터링 결과를 후처리하는 데 사용됩니다.
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(), # 기준선 통계를 지정해서 이 통계는 모델의 정상 동작 범위를 정의합니다.
    constraints=my_default_monitor.suggested_constraints(), # 제안된 제약 조건을 지정합니다. 이 제약 조건은 모니터링 시 모델의 성능을 검증하는 데 사용됩니다.
    schedule_cron_expression=CronExpressionGenerator.hourly(), # 모니터링 작업을 매시간 실행하도록 설정하는 크론 표현식을 생성합니다. 
    enable_cloudwatch_metrics=True, # CloudWatch 메트릭스를 활성화해서 기록합니다. 
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2024-07-22-20-55-19


#### Start generating some artificial traffic

아래의 셀은 엔드포인트로 인위적인 트래픽을 보내기 위해 스레드를 시작합니다.


In [21]:
from threading import Thread
from time import sleep

endpoint_name = predictor.endpoint
runtime_client = sm_session.sagemaker_runtime_client

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, "r") as f:
        for row in f:
            payload = row.rstrip("\n")
            response = runtime_client.invoke_endpoint(
                EndpointName=ep_name, ContentType="text/csv", Body=payload
            )
            response["Body"].read()
            time.sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test_data/test-dataset-input-cols.csv", runtime_client)
        except runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### Describe and inspect the schedule

Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.


In [22]:
desc_schedule_result = my_default_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

Schedule status: Scheduled


#### List executions (실행 x)

MonitoringScheduleStatus은 이전에 지정한 간격에 따라 작업을 시작합니다.
실행을 확인하려면 UTC 시간 기준으로 한 시간이 경과할 때까지 기다려야 할 수 있습니다.
Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.


In [ ]:
mon_executions = my_default_monitor.list_executions()
print(
    "We created a hourly schedule above that begins executions ON the hour (plus 0-20 min buffer.\nWe will have to wait till we hit the hour..."
)

while len(mon_executions) == 0:
    print("Waiting for the first execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()

#### Inspect a specific execution (latest execution)

최신 완료된 또는 실패한 예약 실행을 확인했습니다. 여기서는 가능한 최종 상태와 각 상태의 의미를 설명합니다:

- Completed - 모니터링 실행이 완료되었으며, 위반 보고서에서 문제를 발견하지 못한 상태입니다.
- CompletedWithViolations - 모니터링 실행이 완료되었지만, 제약 조건 위반이 발견된 상태입니다.
- Failed - 모니터링 실행이 실패했습니다. 이는 클라이언트 오류(예: 잘못된 역할 권한) 또는 인프라 문제 때문일 수 있습니다. FailureReason과 ExitMessage를 추가로 조사하여 정확한 원인을 파악해야 합니다.
- Stopped - 작업이 최대 실행 시간을 초과했거나 수동으로 중지된 상태입니다.


In [ ]:
latest_execution = mon_executions[-1]  # Latest execution's index is -1, second to last is -2, etc
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()["ProcessingJobStatus"]))
print("Latest execution result: {}".format(latest_execution.describe()["ExitMessage"]))

latest_job = latest_execution.describe()
if latest_job["ProcessingJobStatus"] != "Completed":
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

In [22]:
report_uri = latest_execution.output.destination
print("Report Uri: {}".format(report_uri))

Report Uri: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2022-04-18-00-13-16/DEMO-xgb-churn-pred-model-monitor-schedule-2022-04-18-00-29-48/2022/04/18/01


#### List the generated reports


In [23]:
from urllib.parse import urlparse

s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip("/")
print("Report bucket: {}".format(report_bucket))
print("Report key: {}".format(report_key))

s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-us-west-2-521695447989
Report key: sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2022-04-18-00-13-16/DEMO-xgb-churn-pred-model-monitor-schedule-2022-04-18-00-29-48/2022/04/18/01
Found Report Files:
sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2022-04-18-00-13-16/DEMO-xgb-churn-pred-model-monitor-schedule-2022-04-18-00-29-48/2022/04/18/01/constraint_violations.json
 sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2022-04-18-00-13-16/DEMO-xgb-churn-pred-model-monitor-schedule-2022-04-18-00-29-48/2022/04/18/01/constraints.json
 sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2022-04-18-00-13-16/DEMO-xgb-churn-pred-model-monitor-schedule-2022-04-18-00-29-48/2022/04/18/01/statistics.json


#### Violations report


Any violations compared to the baseline are listed below.


In [24]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option("display.max_colwidth", None)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,feature_name,constraint_check_type,description
0,Area Code_408,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
1,VMail Plan_no,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
2,State_NV,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
3,State_NC,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
4,State_SC,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
5,State_UT,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
6,State_LA,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
7,State_CA,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
8,State_RI,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.64592817400101% of data is Integral."
9,Churn,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


#### Other commands

We can also start and stop the monitoring schedules.


In [28]:
my_default_monitor.stop_monitoring_schedule()
# my_default_monitor.start_monitoring_schedule()

INFO:sagemaker:Stopping Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2024-07-22-20-55-19


## Delete resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

You need to delete the schedule before deleting the model and endpoint.


In [29]:
my_default_monitor.stop_monitoring_schedule()
my_default_monitor.delete_monitoring_schedule()
time.sleep(60)  # Wait for the deletion

INFO:sagemaker:Stopping Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2024-07-22-20-55-19
INFO:sagemaker:Deleting Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2024-07-22-20-55-19
INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2024-07-22-20-55-19-707


In [30]:
predictor.delete_model()

INFO:sagemaker:Deleting model with name: sagemaker-xgboost-2024-07-22-20-26-07-910


In [31]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07
INFO:sagemaker:Deleting endpoint with name: DEMO-xgb-churn-pred-model-monitor-2024-07-22-20-26-07
